# Example Map Plotting

### At the start of a Jupyter notebook you need to import all modules that you will use

In [ ]:
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata
import cartopy.crs as ccrs                 # For plotting maps
import cartopy.feature as cfeature         # For plotting maps
from cartopy.util import add_cyclic_point  # For plotting maps
import datetime

### Define the directories and file of interest for your results. This can be shortened to less lines as well.

In [ ]:
result_dir = "https://github.com/NCAR/CAM-chem/tree/master/docs/data"
file = "CAM_chem_merra2_FCSD_1deg_QFED_monthoutput_201801.nc"
#the netcdf file is now held in an xarray dataset named 'nc' and can be referenced later in the notebook
nc_load = xr.open_dataset(result_dir+file)
#to see what the netCDF file contains, just call the variable you read it into
nc_load

### Extract the variable of choice at the time and level of choice

In [ ]:
#extract grid variables
lat = nc_load['lat']
lon = nc_load['lon']

#extract variable
var_sel = nc_load['CO']
print(var_sel)

#select the surface level at a specific time and convert to ppbv from vmr
#var_srf = var_sel.isel(time=0, lev=55)
#select the surface level for an average over three times and convert to ppbv from vmr
var_srf = var_sel.isel(time=[0,1,2], lev=55)
var_srf = var_srf.mean('time')
var_srf = var_srf*1e9
print(var_srf.shape)

In [ ]:
# Add cyclic point to avoid white line over Africa
var_srf_cyc, lon_cyc = add_cyclic_point(var_srf, coord=lon) 

### Plot the value over a specific region

In [ ]:
plt.figure(figsize=(20,8))

#Define projection
ax = plt.axes(projection=ccrs.PlateCarree())

#define contour levels
clev = np.arange(0, 100, 1)

#plot the data
plt.contourf(lon_cyc,lat,var_srf_cyc,clev,cmap='Spectral_r')

# add coastlines
ax.add_feature(cfeature.COASTLINE)

#add lat lon grids
ax.gridlines(draw_labels=True, color='grey', alpha=0.5, linestyle='--')

#longitude limits in degrees
ax.set_xlim(20,120)
#latitude limits in degrees
ax.set_ylim(5,60)

# Title
plt.title("CAM-chem 2019 O$_{3}$")

#axes
# y-axis
ax.text(-0.09, 0.55, 'Latitude', va='bottom', ha='center',
        rotation='vertical', rotation_mode='anchor',
        transform=ax.transAxes)
# x-axis
ax.text(0.5, -0.10, 'Longitude', va='bottom', ha='center',
        rotation='horizontal', rotation_mode='anchor',
        transform=ax.transAxes)
# legend
ax.text(1.18, 0.5, 'O$_{3}$ (ppb)', va='bottom', ha='center',
        rotation='vertical', rotation_mode='anchor',
        transform=ax.transAxes)

plt.colorbar()
plt.show() 

### Add location markers

In [ ]:
##Now lets look at the sufrace plot again, but this time add markers for observations at several points.
#first we need to define our observational data into an array
#this can also be imported from text files using various routines
# Kyzylorda, Urzhar, Almaty, Balkhash
obs_lat = np.array([44.8488,47.0870,43.2220,46.2161])
obs_lon = np.array([65.4823,81.6315,76.8512,74.3775])
obs_names = ["Kyzylorda", "Urzhar", "Almaty", "Balkhash"]

In [ ]:
plt.figure(figsize=(20,8))

#Define projection
ax = plt.axes(projection=ccrs.PlateCarree())

#define contour levels
clev = np.arange(0, 100, 1)

#plot the data
plt.contourf(lon_cyc,lat,var_srf_cyc,clev,cmap='Spectral_r')

# add coastlines
ax.add_feature(cfeature.COASTLINE)

#add lat lon grids
ax.gridlines(draw_labels=True, color='grey', alpha=0.5, linestyle='--')

#longitude limits in degrees
ax.set_xlim(20,120)
#latitude limits in degrees
ax.set_ylim(5,60)

# Title
plt.title("CAM-chem 2019 O$_{3}$")

#axes
# y-axis
ax.text(-0.09, 0.55, 'Latitude', va='bottom', ha='center',
        rotation='vertical', rotation_mode='anchor',
        transform=ax.transAxes)
# x-axis
ax.text(0.5, -0.10, 'Longitude', va='bottom', ha='center',
        rotation='horizontal', rotation_mode='anchor',
        transform=ax.transAxes)
# legend
ax.text(1.18, 0.5, 'O$_{3}$ (ppb)', va='bottom', ha='center',
        rotation='vertical', rotation_mode='anchor',
        transform=ax.transAxes)

#convert your observation lat/lon to Lambert-Conformal grid points
#xpt,ypt = m(obs_lon,obs_lat)

#to specify the color of each point it is easiest plot individual points in a loop
for i in range(4):
    plt.plot(obs_lon[i], obs_lat[i], linestyle='none', marker="o", markersize=8, alpha=0.8, c="black", markeredgecolor="black", markeredgewidth=1, transform=ccrs.PlateCarree())
    plt.text(obs_lon[i] - 0.8, obs_lat[i] - 0.5, obs_names[i], horizontalalignment='right', transform=ccrs.PlateCarree())

    
plt.colorbar()
plt.show() 